In [21]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.feature import ShapelyFeature
import cartopy.io.shapereader as shpreader
from cartopy.io.shapereader import Reader
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.feature import (OCEAN, LAKES, BORDERS, COASTLINE, RIVERS, COLORS,
                             LAND)
import netCDF4

#import necessary netcdf files for producing new PM datasets
PM2_5_ds = xr.open_dataset (r'C:\Users\conno\OneDrive\Desktop\WRFChem_Shite\cmip6_PM2_5_processed2014.nc')
PM10_ds = xr.open_dataset (r'C:\Users\conno\OneDrive\Desktop\WRFChem_Shite\cmip6_PM10_processed2014.nc')
OC_ds = xr.open_dataset(r'C:\Users\conno\OneDrive\Desktop\WRFChem_Shite\cmip6_PM2_5_processed2014.nc')
BC_ds = xr.open_dataset(r'C:\Users\conno\OneDrive\Desktop\WRFChem_Shite\cmip6_PM2_5_processed2014.nc')

#Start with primary organic matter
#need to work out scaling factor used to process EDGAR-HTAP2 by sector to figure out what to use for CMIP6
EDGAR_OC = xr.open_dataset(r'F:/edgar_emis/EDGARHTAP2_MEIC2015_OC_2010.0.1x0.1.nc')
EDGAR_POM = xr.open_dataset(r'F:/edgar_emis/EDGARHTAP2_MEIC2015_POM_2010.0.1x0.1.nc')
#print (EDGAR_OC)
#print (EDGAR_POM)

#it is sector specific. I have no agricultural emissions, so likely not bothering with them. Need to muck around with averaging everything
OC_tra = EDGAR_OC['emis_tra'].mean(dim=('lat','lon','time'))
OC_ind = EDGAR_OC['emis_ind'].mean(dim=('lat','lon','time'))
OC_ene = EDGAR_OC['emis_ene'].mean(dim=('lat','lon','time'))
OC_res = EDGAR_OC['emis_res'].mean(dim=('lat','lon','time'))
OC_shp = EDGAR_OC['emis_shp'].mean(dim=('lat','lon','time'))
OC_lto = EDGAR_OC['emis_lto'].mean(dim=('lat','lon','time'))
OC_crs = EDGAR_OC['emis_crs'].mean(dim=('lat','lon','time'))
OC_cds = EDGAR_OC['emis_cds'].mean(dim=('lat','lon','time'))
#will calculate total sepearately later

#now POM
POM_tra = EDGAR_POM['emis_tra'].mean(dim=('lat','lon','time'))
POM_ind = EDGAR_POM['emis_ind'].mean(dim=('lat','lon','time'))
POM_ene = EDGAR_POM['emis_ene'].mean(dim=('lat','lon','time'))
POM_res = EDGAR_POM['emis_res'].mean(dim=('lat','lon','time'))
POM_shp = EDGAR_POM['emis_shp'].mean(dim=('lat','lon','time'))
POM_lto = EDGAR_POM['emis_lto'].mean(dim=('lat','lon','time'))
POM_crs = EDGAR_POM['emis_crs'].mean(dim=('lat','lon','time'))
POM_cds = EDGAR_POM['emis_cds'].mean(dim=('lat','lon','time'))

#print differences
print ("transport scaling factor", POM_tra/OC_tra)
print ("industry scaling factor", POM_ind/OC_ind)
print ("energy scaling factor", POM_ene/OC_ene)
print ("residential scaling factor", POM_res/OC_res)
print ("shipping scaling factor", POM_shp/OC_shp)
print ("landing/takeoff scaling factor", POM_lto/OC_lto)
print ("climbing/descent scaling factor", POM_cds/OC_cds)
print ("cruising scaling factor", POM_crs/OC_crs)

#take these scaling factors and create POM from cmip6 OC
new_POM = OC_ds
new_POM = new_POM.drop_vars('emis_tra')
new_POM['emis_tra'] = OC_ds['emis_tra']*1.4999994
new_POM = new_POM.drop_vars('emis_ind')
new_POM['emis_ind'] = OC_ds['emis_ind']*1.499997
new_POM = new_POM.drop_vars('emis_ene')
new_POM['emis_ene'] = OC_ds['emis_ene']*1.4999924
new_POM = new_POM.drop_vars('emis_res')
new_POM['emis_res'] = OC_ds['emis_res']*1.4999985
new_POM = new_POM.drop_vars('emis_shp')
new_POM['emis_shp'] = OC_ds['emis_shp']*1.5000056
new_POM = new_POM.drop_vars('emis_lto')
new_POM['emis_lto'] = OC_ds['emis_lto']*1.4999965
new_POM = new_POM.drop_vars('emis_cds')
new_POM['emis_cds'] = OC_ds['emis_cds']*1.4999987
new_POM = new_POM.drop_vars('emis_crs')
new_POM['emis_crs'] = OC_ds['emis_crs']*1.4999995

#now get total
new_POM = new_POM.drop_vars('emis_tot')
new_POM['emis_tot'] = new_POM['emis_tra'] + new_POM['emis_ind'] + new_POM['emis_ene'] + new_POM['emis_shp'] + new_POM['emis_lto'] + new_POM['emis_cds'] +new_POM['emis_crs']

#new_POM.to_netcdf("cmip6_POM_processed2014.nc")


#PM2.5_10 is everything that counts as PM10, but does not fall under PM2.5

PM2_5_10 = PM10_ds - PM2_5_ds 
#doing the above turns the date variable into zeroes, so need to remove and readd. Doesn't matter with datesec
PM2_5_10 = PM2_5_10.drop_vars('date')

#readd date

the_time = PM2_5_10['time']

dates=np.array([20140116, 20140216, 20140316, 20140416, 20140516, 20140616, 20140716, 20140816, 20140916, 20141016, 20141116, 20141216], dtype=int) #these are int objects that need to be added too for some reason

date = xr.DataArray(dates, coords=[("time", the_time)]) #read in the same way as datesec

PM2_5_10['date'] = date

print(PM2_5_10)

#convert to netcdf
#PM2_5_10.to_netcdf("cmip6_PM2.5_10_processed2014.nc")

#next is OIN_PM2.5, which is PM2.5 - BC - POM

OIN_PM2_5 = PM2_5_ds - BC_ds - new_POM
#date will need sorted again
OIN_PM2_5 = OIN_PM2_5.drop_vars('date')
OIN_PM2_5['date'] = date #the time is the same in all files, so don't need to recreate that.

#need to set negative values to zero
new_OIN = OIN_PM2_5.where(OIN_PM2_5>0,0) #this in theory replaces everything with zero

print(OIN_PM2_5) #print both for comparative purposes
print(new_OIN)
#ne_OIN.to_netcdf("cmip6_OIN_PM2.5_processed2014.nc")

transport scaling factor <xarray.DataArray 'emis_tra' ()>
array(1.4999994, dtype=float32)
industry scaling factor <xarray.DataArray 'emis_ind' ()>
array(1.499997, dtype=float32)
energy scaling factor <xarray.DataArray 'emis_ene' ()>
array(1.4999924, dtype=float32)
residential scaling factor <xarray.DataArray 'emis_res' ()>
array(1.4999985, dtype=float32)
shipping scaling factor <xarray.DataArray 'emis_shp' ()>
array(1.5000056, dtype=float32)
landing/takeoff scaling factor <xarray.DataArray 'emis_lto' ()>
array(1.4999965, dtype=float32)
climbing/descent scaling factor <xarray.DataArray 'emis_cds' ()>
array(1.4999987, dtype=float32)
cruising scaling factor <xarray.DataArray 'emis_crs' ()>
array(1.4999995, dtype=float32)
<xarray.Dataset>
Dimensions:   (lat: 360, lon: 720, time: 12)
Coordinates:
  * time      (time) object 2014-01-16 00:00:00 ... 2014-12-16 00:00:00
  * lat       (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon       (lon) float64 -179.8 -179.2 -178.